In [87]:
import pandas as pd

In [88]:
import re


In [89]:
df = pd.read_csv('Orthogroups.tsv', sep='\t', low_memory=False)


In [90]:
species = pd.read_csv('important_species_list.txt', header=None)
species_list = species[0].tolist()


In [92]:
print(species_list)

['Acyrthosiphon_pisum', 'Aedes_aegypti', 'Drosophila_mojavensis', 'Bombyx_mori', 'Helicoverpa_armigera', 'Apis_mellifera', 'Blattella_germanica', 'Gryllus_bimaculatus', 'Tribolium_castaneum']


In [93]:
# 生成正则匹配模式（忽略大小写，并允许部分匹配）
species_pattern = "|".join(species_list)  # 组合成正则表达式 'Actias_luna|Adoxophyes_honmai|...'


In [94]:
# 找到所有匹配的列（第一列始终保留）
cols_to_keep = [df.columns[0]] + [col for col in df.columns if re.search(species_pattern, col, re.IGNORECASE)]

In [95]:
# 生成提取后的 DataFrame
df_filtered = df[cols_to_keep]

In [96]:
df_filtered = df_filtered.copy()
df_filtered.drop("Bombyx_mori.protein", axis=1, inplace=True)
df_filtered.rename(columns={"Bombyx_mori_new.protein": "Bombyx_mori.protein"}, inplace=True)

In [97]:
df_filtered.columns = df_filtered.columns.str.replace(r'\.protein', '', regex=True)


In [98]:
print(df_filtered)

       Orthogroup                                Acyrthosiphon_pisum  \
0       OG0000000  XM_008179938.1, XM_008179973.1, XM_008180018.1...   
1       OG0000001  XM_008183679.1, XM_008184658.3, XM_008189338.1...   
2       OG0000002  XM_003241194.1, XM_003245210.4, XM_008180012.1...   
3       OG0000003  XM_003243647.1, XM_003243710.2, XM_003244376.2...   
4       OG0000004  XM_008181907.1, XM_008181926.1, XM_008181927.1...   
...           ...                                                ...   
280541  OG0280541                                                NaN   
280542  OG0280542                                                NaN   
280543  OG0280543                                                NaN   
280544  OG0280544                                                NaN   
280545  OG0280545                                                NaN   

                                            Aedes_aegypti Apis_mellifera  \
0       g10265.t1, g10290.t1, g10291.t1, g10875.t1, g1...  

In [99]:
gene_columns = df_filtered.columns


In [100]:
print(gene_columns)

Index(['Orthogroup', 'Acyrthosiphon_pisum', 'Aedes_aegypti', 'Apis_mellifera',
       'Blattella_germanica', 'Drosophila_mojavensis', 'Gryllus_bimaculatus',
       'Helicoverpa_armigera', 'Tribolium_castaneum', 'Bombyx_mori'],
      dtype='object')


In [101]:
def count_gene_ids(cell):
    """
    统计单元格中基因ID的数量：
      - 对于空值返回 0
      - 按','拆分后，去除前后空格和空字符串，再统计数量
    """
    if pd.isnull(cell):
        return 0
    # 将 cell 转为字符串（防止非字符串类型），按','拆分
    # 使用 strip() 去除可能的空白字符，并过滤掉空串
    ids = [gene.strip() for gene in str(cell).split(',') if gene.strip()]
    return len(ids)

In [102]:
counts_df = df_filtered[gene_columns].map(count_gene_ids)


In [103]:
print(counts_df)

        Orthogroup  Acyrthosiphon_pisum  Aedes_aegypti  Apis_mellifera  \
0                1                  333            358               0   
1                1                   26            238               0   
2                1                   81            178               0   
3                1                   74            264               0   
4                1                   30            125               0   
...            ...                  ...            ...             ...   
280541           1                    0              0               0   
280542           1                    0              0               0   
280543           1                    0              0               0   
280544           1                    0              0               0   
280545           1                    0              0               0   

        Blattella_germanica  Drosophila_mojavensis  Gryllus_bimaculatus  \
0                         4         

In [118]:
valid_rows = counts_df.apply(lambda row: (row == 1).sum() / len(row) >= 0.75, axis=1)


In [119]:
print(valid_rows)

0         False
1         False
2         False
3         False
4         False
          ...  
280541    False
280542    False
280543    False
280544    False
280545    False
Length: 280546, dtype: bool


In [122]:
df_valid = df_filtered[valid_rows]


In [123]:
print(df_valid)

     Orthogroup                                Acyrthosiphon_pisum  \
969   OG0000969  XM_003245443.4, XM_003245444.4, XM_008186260.3...   
1163  OG0001163                                                NaN   
1411  OG0001411                     XM_008187848.2, XM_029492190.1   
1585  OG0001585                                     XM_001952156.4   
1588  OG0001588                                     XM_008188146.2   
...         ...                                                ...   
7895  OG0007895                                     XM_003240088.4   
7947  OG0007947                                     XM_016806733.2   
8009  OG0008009                                     NM_001135904.1   
8033  OG0008033                                     XM_003246949.4   
8133  OG0008133                                     NM_001145431.1   

                        Aedes_aegypti Apis_mellifera  \
969                           g599.t1   Amei005982.1   
1163                        g19837.t1   Amei004

In [124]:
# 输出到文件（同样假设使用制表符分隔，且不保留索引）
df_valid.to_csv('important_species_orthogroups_at_lest_7_species.txt', sep='\t', index=False)